Script 3:
- Down sampling
- Feature selection

In [60]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import balanced_accuracy_score

from imblearn.ensemble import BalancedBaggingClassifier


import scipy.cluster.hierarchy as shc

import seaborn as sns

## Import Data

In [22]:
df_modified = pd.read_csv('df_modified.csv')
display(df_modified.head())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DOCUMENT_SUM
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,2.0,2.0,1134.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,1.0,0.0,828.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,815.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,2.0,0.0,617.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,1106.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [36]:
df_combined = pd.read_csv('Combined.csv')
display(df_combined.head())

,Unnamed: 0,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,STATUS_2_TO_4_PERC,CHILDREN_RATIO,INCOME_PER_CHLD,EMPLOYED_BIRTH_RATIO,LOAN_INCOME_RATIO,ANNUITY_LENGTH,CAR_TO_BIRTH_RATIO,OWN_CAR_AGE_TO_EMPLOY_RATIO,INCOME_PER_PERSON,ANNUITY_INCOME_PERC
0,0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,...,0.0,0.0,202500.0,0.067329,2.007889,16.461104,NaN,NaN,12.218500,0.349252
1,1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,...,0.0,0.0,270000.0,0.070862,4.790750,36.234085,NaN,NaN,11.813037,0.363615
2,2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,...,0.0,0.0,67500.0,0.011814,2.000000,20.000000,-0.001365,-0.115556,11.119898,0.316225
3,3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,...,NaN,0.0,135000.0,0.159905,2.316167,10.532818,NaN,NaN,11.119898,0.468933
4,4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,...,0.0,0.0,121500.0,0.152418,4.222222,23.461618,NaN,NaN,11.707678,0.424219


In [37]:
selected_feature = pd.read_csv('random_forest_selection 2.csv')
df_combined = df_combined[[col for col in df_combined.columns if col in selected_feature['0'].tolist()]]

display(df_combined.head())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,STATUS_0_PERC,STATUS_C_PERC,STATUS_X_PERC,CHILDREN_RATIO,INCOME_PER_CHLD,EMPLOYED_BIRTH_RATIO,LOAN_INCOME_RATIO,ANNUITY_LENGTH,INCOME_PER_PERSON,ANNUITY_INCOME_PERC
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.409091,0.209091,0.136364,0.0,202500.0,0.067329,2.007889,16.461104,12.218500,0.349252
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,1.000000,0.0,270000.0,0.070862,4.790750,36.234085,11.813037,0.363615
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,1.000000,0.0,67500.0,0.011814,2.000000,20.000000,11.119898,0.316225
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,0.0,135000.0,0.159905,2.316167,10.532818,11.119898,0.468933
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.000000,0.000000,1.000000,0.0,121500.0,0.152418,4.222222,23.461618,11.707678,0.424219


## Data Cleaning

#### df_modified.csv

In [10]:
# For df_modified.csv

# Change categorical variable
ordinal_encoder = OrdinalEncoder()
cat_encoded = ordinal_encoder.fit_transform(df_modified.select_dtypes('object'))
df_cat_encoded = pd.DataFrame(cat_encoded, columns = df_modified.select_dtypes('object').columns)

# Normalize numeric variables
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num = df_modified.select_dtypes(include = numerics)
df_col = df_num.columns[2:]
transformer = Normalizer().fit(df_num.iloc[:,2:])
df_index = df_num.iloc[:,0:2]
df_num = pd.DataFrame(transformer.transform(df_num.iloc[:,2:]), columns = df_col)

# Combine Data
df_num = pd.concat([df_index, df_num], axis=1, sort = False)
df_modified = pd.concat([df_num, df_cat_encoded], axis=1, sort=False) 

display(df_modified.head())

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,AGE,DAYS_EMPLOYED,...,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE
0,100002,1,0.0,0.352423,0.707626,0.042988,0.610866,3.272049e-08,0.000045,0.001109,...,0.0,1.0,6.0,7.0,4.0,3.0,1.0,8.0,6.0,5.0
1,100003,0,0.0,0.155288,0.743946,0.020532,0.649621,2.036573e-09,0.000026,0.000683,...,0.0,0.0,1.0,4.0,1.0,1.0,1.0,3.0,1.0,39.0
2,100004,0,0.0,0.333046,0.666092,0.033305,0.666092,4.949801e-08,0.000257,0.001110,...,1.0,1.0,6.0,7.0,4.0,3.0,1.0,8.0,1.0,11.0
3,100006,0,0.0,0.298021,0.690267,0.065535,0.655647,1.770247e-08,0.000115,0.006709,...,0.0,1.0,6.0,7.0,4.0,0.0,1.0,8.0,6.0,5.0
4,100007,0,0.0,0.165093,0.697060,0.029711,0.697060,3.894705e-08,0.000075,0.004128,...,0.0,1.0,6.0,7.0,4.0,3.0,1.0,3.0,4.0,37.0


#### Combined.csv

In [38]:
# For Combined.csv

# Solve for Infinite Value
df_combined['AMT_CREDIT_SUM_RATIO'][df_combined['AMT_CREDIT_SUM_RATIO']>1000000] = df_combined['AMT_CREDIT_SUM_RATIO'].median()

def missingval(data):

    miss_tot = data.isnull().sum()
    miss_perc = data.isnull().sum()/len(data)
    miss_table = pd.concat([miss_tot,miss_perc],axis = 1, keys = ['miss_tot','miss_perc']).sort_values(by = 'miss_perc',ascending = False)
    miss_table_unique = miss_table.drop_duplicates('miss_tot')
    return miss_table,miss_table_unique

# missing variables
df_miss,df_miss_u = missingval(df_combined)
#drop missing data greater than 40%
df_combined.drop(df_miss[df_miss.miss_perc > 0.4].index,axis = 1,inplace = True)
#miss_app[miss_app.miss_perc > 0.4]
df_miss.head()

df_combined.STATUS_X_PERC.fillna(1,inplace = True)
df_combined.STATUS_C_PERC.fillna(0,inplace = True)
df_combined.STATUS_0_PERC.fillna(0,inplace = True)
#df_combined.STATUS_1_PERC.fillna(0,inplace = True)
#df_combined.STATUS_5_PERC.fillna(0,inplace = True)
#df_combined.STATUS_2_TO_4_PERC.fillna(0,inplace = True)

#use median to fill the rest of numeric feature
df_combined.fillna(df_combined.median(),inplace=True)
df_combined.fillna('XNA',inplace=True)

In [39]:
df_combined = df_combined.loc[:,~df_combined.columns.str.startswith('TARGET_')]

display(df_combined.head())
display(df_combined.shape)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,STATUS_0_PERC,STATUS_C_PERC,STATUS_X_PERC,CHILDREN_RATIO,INCOME_PER_CHLD,EMPLOYED_BIRTH_RATIO,LOAN_INCOME_RATIO,ANNUITY_LENGTH,INCOME_PER_PERSON,ANNUITY_INCOME_PERC
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.409091,0.209091,0.136364,0.0,202500.0,0.067329,2.007889,16.461104,12.218500,0.349252
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,1.000000,0.0,270000.0,0.070862,4.790750,36.234085,11.813037,0.363615
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,1.000000,0.0,67500.0,0.011814,2.000000,20.000000,11.119898,0.316225
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.000000,0.000000,1.000000,0.0,135000.0,0.159905,2.316167,10.532818,11.119898,0.468933
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.000000,0.000000,1.000000,0.0,121500.0,0.152418,4.222222,23.461618,11.707678,0.424219


(307511, 262)

In [41]:
# For Combined.csv

# Change categorical variable
ordinal_encoder = OrdinalEncoder()
cat_encoded = ordinal_encoder.fit_transform(df_combined.select_dtypes('object'))
df_cat_encoded = pd.DataFrame(cat_encoded, columns = df_combined.select_dtypes('object').columns)

# Normalize numeric variables
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num = df_combined.select_dtypes(include = numerics)
df_col = df_num.columns[2:]
transformer = Normalizer().fit(df_num.iloc[:,2:])
df_index = df_num.iloc[:,0:2]
df_num = pd.DataFrame(transformer.transform(df_num.iloc[:,2:]), columns = df_col)

# Combine Data
df_num = pd.concat([df_index, df_num], axis=1, sort = False)
df_combined = pd.concat([df_num, df_cat_encoded], axis=1, sort=False) 

display(df_combined.head())

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,NAME_CLIENT_TYPE_Pre,NAME_GOODS_CATEGORY_Pre,NAME_PORTFOLIO_Pre,NAME_PRODUCT_TYPE_Pre,CHANNEL_TYPE_Pre,NAME_SELLER_INDUSTRY_Pre,NAME_YIELD_GROUP_Pre,PRODUCT_COMBINATION_Pre,CREDIT_ACTIVE_MAX,CREDIT_TYPE_MAX
0,100002,1,0.0,0.191107,0.383721,0.023311,0.331252,-0.000601,-0.003443,-0.002001,...,0.0,23.0,3.0,0.0,7.0,0.0,29.0,15.0,2.0,3.0
1,100003,0,0.0,0.061659,0.295393,0.008152,0.257940,-0.000271,-0.000271,-0.000066,...,1.0,1361.0,3.0,0.0,107.0,243.0,30.0,3605.0,2.0,3.0
2,100004,0,0.0,0.270375,0.540750,0.027037,0.540750,-0.000901,-0.017064,-0.010138,...,0.0,17.0,3.0,0.0,6.0,2.0,30.0,14.0,2.0,2.0
3,100006,0,0.0,0.025084,0.058099,0.005516,0.055185,-0.000565,-0.001827,-0.000453,...,2.0,25.0,27.0,0.0,5.0,10.0,0.0,4315.0,4.0,12.0
4,100007,0,0.0,0.055153,0.232867,0.009925,0.232867,-0.001379,-0.001957,-0.001570,...,2.0,25.0,2.0,6.0,4.0,4.0,22.0,8.0,2.0,2.0


## Random Forest: Based Model

In [42]:
df_modified = df_combined

In [ ]:
# 1. Data Prepariation
Y = np.array(df_modified['TARGET'])
X = df_modified.drop('TARGET', axis = 1).iloc[:, 1:]
feature = list(X.columns)
X = np.array(X) 

# 2. Split data into training and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42) 

# 3. Train the Random Forest on Training Data
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42, bootstrap=True, n_jobs=-1, min_samples_split=20, min_samples_leaf=10)
rf.fit(X_train, Y_train)
Y_pred = rf.predict(X_test)

In [10]:
print("The AUC score is {}.".format(metrics.roc_auc_score(Y_test, Y_pred)))
print("AUC = 0.5 therefore the model guessess the Target randomly")

The AUC score is 0.5.
AUC = 0.5 therefore the model guessess the Target randomly


In [11]:
confusion_matrix(Y_test, Y_pred)

array([[93362,     0],
       [ 8117,     0]])

In [15]:
importance = list(rf.feature_importances_)
feature_importance = pd.DataFrame({'feature': feature,'importance': importance})
feature_importance = feature_importance.sort_values('importance', ascending=False)
feature_importance.head(10)

,feature,importance
241,AMT_CREDIT_SUM_DEBT_MAX,0.013422
3,AMT_ANNUITY,0.012938
234,DAYS_CREDIT_MEAN,0.011972
7,DAYS_EMPLOYED,0.011290
94,CNT_PAYMENT_Pre_sum,0.011025
88,AMT_CREDIT_Pre_sum,0.010967
116,AMT_PAYMENT_Ins_sum,0.010595
115,AMT_INSTALMENT_Ins_sum,0.010583
236,DAYS_CREDIT_MAX,0.010478
89,AMT_GOODS_PRICE_Pre_sum,0.010448


## Random Forest: Down Sampling Train Data

### Method 1

#### Fix the Test Data

In [ ]:
df_modified = df_combined

In [51]:
# Count the number of Target 0 & 1
count0, count1 = df_modified.TARGET.value_counts()

# Split the Target 0 & 1
df_class0 = df_modified[df_modified['TARGET'] == 0]
df_class1 = df_modified[df_modified['TARGET'] == 1]

# Split data into training and test set for df_class0 and df_class1
df_train_0, df_test_0 = train_test_split(df_class0, test_size=0.33, random_state=42)
df_train_1, df_test_1 = train_test_split(df_class1, test_size=0.33, random_state=42)

# Combine to form the whole df_train and df_test
df_train = pd.concat([df_train_0, df_train_1])
df_test = pd.concat([df_test_0, df_test_1])

# Data Prepariation: Fix Test Data
Y_test = np.array(df_test['TARGET'])
X_test = df_test.drop('TARGET', axis = 1).iloc[:, 1:]
feature = list(X_test.columns)
X_test = np.array(X_test)

#### Down Sampling the Train Data

In [45]:
# Count the number of train data with Target 1
count1_train = df_train_1.shape[0]

# Store each random forest's result
auc = []
confusion = []
importance = []

In [46]:
# n_estimators = 1000
# Down Sampling for 10 times
for i in range(10):
    # 1. Down Sampling Train Data: Balance Train Data
    df_train_0_down = df_train_0.sample(count1_train)

    # Merge the balance train data
    df_train_down = pd.concat([df_train_0_down, df_train_1], axis = 0)

    # 2. Data Prepariation: Train Data
    Y_train = np.array(df_train_down['TARGET'])
    X_train = df_train_down.drop('TARGET', axis = 1).iloc[:, 1:]
    X_train = np.array(X_train)

    # 3. Train the Random Forest on Training Data of Down Sampling
    rf = RandomForestClassifier(n_estimators = 1000, random_state = 42, bootstrap=True, n_jobs=-1, min_samples_split=20, min_samples_leaf=10)
    rf.fit(X_train, Y_train)
    Y_pred = rf.predict(X_test)

    a = metrics.roc_auc_score(Y_test, Y_pred)
    auc.append(a)

    c = confusion_matrix(Y_test, Y_pred)
    confusion.append(c)
    
    i = list(rf.feature_importances_)
    importance.append(i)

In [47]:
print("The average AUC score is {}.".format(np.average(auc, axis=0)))

The average AUC score is 0.6561017992466263.


In [48]:
np.average(confusion, axis=0)

array([[62036.3, 31250.7],
       [ 2890.5,  5302.5]])

In [18]:
feature_importance = pd.DataFrame({'feature': feature,'importance': np.average(importance, axis=0)})
feature_importance = feature_importance.sort_values('importance', ascending=False)

### Method 2: imblearn

In [57]:
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(), sampling_strategy='auto', replacement=False, random_state=0)

bbc.fit(X_train, Y_train)

Y_pred = bbc.predict(X_test)

In [61]:
balanced_accuracy_score(Y_test, Y_pred) 

0.6040116234874463